In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from utils.meters import AverageMeter, accuracy

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 40
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./cifar10',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./cifar10',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,shuffle=False) 


In [3]:
                                          

# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        #TODO
        # 1) convert x(in float) to posit
        # 2) convert back posit to float and store in x
        # 3) repeat the same for all the inputs of conv functions
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        #TODO
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            #TODO - apply conversion from float to posit and then to float of x in below line
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
         #TODO
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        #TODO
        out = self.layer1(out)

        #TODO
        out = self.layer2(out)

        #TODO
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
model

ResNet(
  (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=

In [4]:

num_parameters = sum([l.nelement() for l in model.parameters()])
print(num_parameters)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        prec1, prec5 = accuracy(outputs.detach(), labels, topk=(1, 5))
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f} Top 1: {} Top 5: {}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item(),prec1,prec5))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)



195738
Epoch [1/40], Step [100/500] Loss: 1.5788 Top 1: 44.0 Top 5: 89.0
Epoch [1/40], Step [200/500] Loss: 1.3178 Top 1: 53.0 Top 5: 94.0
Epoch [1/40], Step [300/500] Loss: 1.3418 Top 1: 52.0 Top 5: 91.0
Epoch [1/40], Step [400/500] Loss: 1.2328 Top 1: 55.0 Top 5: 96.0
Epoch [1/40], Step [500/500] Loss: 1.1940 Top 1: 59.0 Top 5: 98.0
Epoch [2/40], Step [100/500] Loss: 1.2657 Top 1: 58.0 Top 5: 95.0
Epoch [2/40], Step [200/500] Loss: 1.0963 Top 1: 58.0 Top 5: 97.0
Epoch [2/40], Step [300/500] Loss: 1.0623 Top 1: 61.0 Top 5: 96.0
Epoch [2/40], Step [400/500] Loss: 0.8725 Top 1: 66.0 Top 5: 98.0
Epoch [2/40], Step [500/500] Loss: 0.8634 Top 1: 65.0 Top 5: 99.0
Epoch [3/40], Step [100/500] Loss: 1.0047 Top 1: 61.0 Top 5: 96.0
Epoch [3/40], Step [200/500] Loss: 0.6753 Top 1: 76.0 Top 5: 98.0
Epoch [3/40], Step [300/500] Loss: 0.7319 Top 1: 73.0 Top 5: 99.0
Epoch [3/40], Step [400/500] Loss: 0.6651 Top 1: 78.0 Top 5: 100.0
Epoch [3/40], Step [500/500] Loss: 0.7196 Top 1: 79.0 Top 5: 99.0
Ep

In [8]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
# torch.save(model.state_dict(), 'resnet.ckpt')

Accuracy of the model on the test images: 86.64 %


In [9]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            432
├─BatchNorm2d: 1-2                       32
├─ReLU: 1-3                              --
├─Sequential: 1-4                        --
│    └─ResidualBlock: 2-1                --
│    │    └─Conv2d: 3-1                  2,304
│    │    └─BatchNorm2d: 3-2             32
│    │    └─ReLU: 3-3                    --
│    │    └─Conv2d: 3-4                  2,304
│    │    └─BatchNorm2d: 3-5             32
│    └─ResidualBlock: 2-2                --
│    │    └─Conv2d: 3-6                  2,304
│    │    └─BatchNorm2d: 3-7             32
│    │    └─ReLU: 3-8                    --
│    │    └─Conv2d: 3-9                  2,304
│    │    └─BatchNorm2d: 3-10            32
├─Sequential: 1-5                        --
│    └─ResidualBlock: 2-3                --
│    │    └─Conv2d: 3-11                 4,608
│    │    └─BatchNorm2d: 3-12            64
│    │    └

In [10]:
from torchsummary import summary
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 16, 32, 32]           2,304
       BatchNorm2d-5           [-1, 16, 32, 32]              32
              ReLU-6           [-1, 16, 32, 32]               0
            Conv2d-7           [-1, 16, 32, 32]           2,304
       BatchNorm2d-8           [-1, 16, 32, 32]              32
              ReLU-9           [-1, 16, 32, 32]               0
    ResidualBlock-10           [-1, 16, 32, 32]               0
           Conv2d-11           [-1, 16, 32, 32]           2,304
      BatchNorm2d-12           [-1, 16, 32, 32]              32
             ReLU-13           [-1, 16, 32, 32]               0
           Conv2d-14           [-1, 16,